In [1]:
%pip install --quiet --upgrade pip 
%pip install numpy --quiet
%pip install Pandas --quiet
%pip install sklearn --quiet
%pip install ipywidgets --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Horse Racing Results Analysis using Linear regression #

Use simple linear regression techniques to determine if we can reliably predict the **speed** of a horse in a given race based on its previous performance. 

Data used here is derived from the features extracted by the [Feature Analysis](https://github.com/LeeSanderson/RacingData/blob/main/Data/FeatureAnalysis.ipynb) notebook.

In [2]:
import numpy as np
import pandas as pd
import math
from abc import ABC, abstractmethod
from datetime import datetime, date

In [3]:
races = pd.read_csv("Race_Features.csv")
races['Off'] =  pd.to_datetime(races['Off'], format='%Y-%m-%d %H:%M:%S')
races.columns

Index(['RaceId', 'CourseId', 'RaceType', 'Off', 'DecimalOdds',
       'OfficialRating', 'RacingPostRating', 'TopSpeedRating',
       'DistanceInMeters', 'Going', 'Surface', 'HorseId', 'HorseName',
       'JockeyId', 'JockeyName', 'Age', 'HeadGear', 'RaceCardNumber',
       'StallNumber', 'WeightInPounds', 'FinishingPosition',
       'OverallBeatenDistance', 'RaceTimeInSeconds', 'Wins',
       'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 'Going_Firm',
       'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy',
       'Going_Soft', 'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other',
       'RaceType_SteepleChase', 'Speed', 'HorseCount', 'KnownHorseAndJockey',
       'NumberOfPriorRaces', 'LastRaceGoing', 'LastRaceSurface',
       'LastRaceDistanceInMeters', 'LastRaceWeightInPounds', 'LastRaceSpeed',
       'DaysRested', 'LastRaceDecimalOdds', 'LastRaceOfficialRating',
       'LastRaceRacingPostRating', 'LastRaceTopSpeedRating',
       'LastRaceAvgRelFinishi

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

race_info = ['RaceId', 'CourseId', 'Off', 'HorseId', 'HorseName', 'HorseCount', 'JockeyId', 'JockeyName', 'DecimalOdds', 'Wins', 'FinishingPosition']
predictors = ([
    'DistanceInMeters', 
    'WeightInPounds', 
    'Surface_AllWeather', 'Surface_Dirt', 'Surface_Turf', 
    'Going_Firm', 'Going_Good', 'Going_Good_To_Firm', 'Going_Good_To_Soft', 'Going_Heavy', 'Going_Soft', 
    'RaceType_Flat', 'RaceType_Hurdle', 'RaceType_Other', 'RaceType_SteepleChase', 

    'LastRaceDistanceInMeters', 
    'LastRaceWeightInPounds', 
    'LastRaceSpeed',
    'DaysRested',
    'LastRaceAvgRelFinishingPosition', 
    'LastRaceSurface_AllWeather', 'LastRaceSurface_Dirt', 'LastRaceSurface_Turf', 
    'LastRaceGoing_Good', 'LastRaceGoing_Good_To_Soft', 'LastRaceGoing_Soft', 'LastRaceGoing_Good_To_Firm', 'LastRaceGoing_Firm', 'LastRaceGoing_Heavy', 
    'LastRaceRaceType_Other', 'LastRaceRaceType_Hurdle', 'LastRaceRaceType_SteepleChase', 'LastRaceRaceType_Flat', 
    
    'JockeyNumberOfPriorRaces',
    'DaysSinceJockeyLastRaced',     
    'JockeyWinPercentage',
    'JockeyTop3Percentage',
    'JockeyAvgRelFinishingPosition'
    ])
prediction = ["Speed"]

train = races[race_info + predictors + prediction].dropna().copy()

In [5]:
# Cap rested days
train.loc[train["DaysRested"] > 10, "DaysRested"] = 10
train.loc[train["DaysSinceJockeyLastRaced"] > 10, "DaysSinceJockeyLastRaced"] = 10

# Cap prior races
# train.loc[train["JockeyNumberOfPriorRaces"] > 400] = 400 (negative effect)

train[predictors].describe()

,DistanceInMeters,WeightInPounds,Surface_AllWeather,Surface_Dirt,Surface_Turf,Going_Firm,Going_Good,Going_Good_To_Firm,Going_Good_To_Soft,Going_Heavy,Going_Soft,RaceType_Flat,RaceType_Hurdle,RaceType_Other,RaceType_SteepleChase,LastRaceDistanceInMeters,LastRaceWeightInPounds,LastRaceSpeed,DaysRested,LastRaceAvgRelFinishingPosition,LastRaceSurface_AllWeather,LastRaceSurface_Dirt,LastRaceSurface_Turf,LastRaceGoing_Good,LastRaceGoing_Good_To_Soft,LastRaceGoing_Soft,LastRaceGoing_Good_To_Firm,LastRaceGoing_Firm,LastRaceGoing_Heavy,LastRaceRaceType_Other,LastRaceRaceType_Hurdle,LastRaceRaceType_SteepleChase,LastRaceRaceType_Flat,JockeyNumberOfPriorRaces,DaysSinceJockeyLastRaced,JockeyWinPercentage,JockeyTop3Percentage,JockeyAvgRelFinishingPosition
count,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000,287651.000000
mean,2272.064846,153.929112,0.236418,0.018324,0.745212,0.027652,0.464448,0.094006,0.224453,0.068357,0.120876,0.006956,0.185287,0.729127,0.078630,2236.321925,153.792825,15.120926,9.764440,0.529883,0.232622,0.017893,0.749405,0.462877,0.222558,0.120643,0.098348,0.026824,0.068545,0.731456,0.183903,0.072974,0.011667,404.830600,3.036930,0.110641,0.327870,0.551123
std,1146.069742,15.894508,0.424883,0.134121,0.435743,0.163973,0.498735,0.291838,0.417222,0.252358,0.325983,0.083114,0.388531,0.444412,0.269161,1125.197669,15.818139,1.527127,0.992077,0.192903,0.422504,0.132564,0.433357,0.498621,0.415965,0.325712,0.297786,0.161570,0.252679,0.443203,0.387406,0.260094,0.107382,401.153012,2.799696,0.063783,0.124623,0.069980
min,804.000000,119.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,804.000000,119.000000,9.763058,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.052632
25%,1407.000000,144.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1407.000000,144.000000,13.809537,10.000000,0.402303,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,104.000000,1.000000,0.071429,0.245227,0.511172
50%,1809.000000,149.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1709.000000,149.000000,15.555013,10.000000,0.519231,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,277.000000,2.000000,0.103659,0.324224,0.546862
75%,3217.000000,165.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,3217.000000,164.000000,16.270912,10.000000,0.648451,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,578.000000,4.000000,0.142857,0.405720,0.589492
max,6937.000000,200.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,6937.000000,199.000000,19.670069,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2441.000000,10.000000,1.000000,1.000000,1.000000


In [6]:
# Now that we've dropped any rows with na values, calculate which races in which we can predict the speed for all horses.
train = train.drop("PredictableHorseCount", axis=1, errors='ignore')
groups = train.groupby(['RaceId']).apply(lambda g: pd.Series({'PredictableHorseCount': g['RaceId'].count()}, index=['PredictableHorseCount']))
train = pd.merge(train, groups, how='left', on=['RaceId'])

# 50% of races have 11 horse, 25% have 8 or less, 14% have 6 or less, 9% have 5 or less. Fewer horses should be more predictable
all_races_count = len(races["RaceId"].unique())
all_predictable = train[(train["HorseCount"] == train["PredictableHorseCount"]) & (train["HorseCount"] <= 5)]["RaceId"].unique().tolist()
all_predictable_count = len(all_predictable)
print(f"Possible predictable races = {all_predictable_count} out of {all_races_count} ({all_predictable_count * 100 / all_races_count}%)")

Possible predictable races = 4035 out of 37935 (10.636615262949782%)


In [7]:
from sklearn.model_selection import train_test_split

_, test_race_ids = train_test_split(all_predictable, test_size=0.2, random_state = 42)

test = train[train["RaceId"].isin(test_race_ids)]
train = train[train["RaceId"].isin(test_race_ids) == False]

In [19]:
# Use Ridge over standard ordinary least squares LinearRegression as more robust to colinearity
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
# from sklearn.linear_model import Lasso doesn't work - all horses get same predicted speed
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_squared_error

# Create a pipeline that includes feature scaling and linear regression
model = make_pipeline(StandardScaler(), PolynomialFeatures(degree=2, interaction_only=False), Ridge())

In [20]:
#help(model.fit)

In [21]:
inputs = train[predictors]
targets = train[prediction[0]].values

# print(inputs.shape)
# print(targets.shape)

In [22]:
model.fit(inputs, targets)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('polynomialfeatures', PolynomialFeatures()),
                ('ridge', Ridge())])

In [23]:
train_predictions = model.predict(inputs)
train_mse = mean_squared_error(targets, train_predictions)
train_rmse = np.sqrt(train_mse)
train_rmse

0.47957849348329956

In [24]:
test_inputs = test[predictors]
predictions = model.predict(test_inputs)
test_with_predictions = test.copy()
test_with_predictions["PredictedSpeed"] = predictions

test_mse = mean_squared_error(test_with_predictions["Speed"], predictions)
test_rmse = np.sqrt(test_mse)
test_rmse

0.46221726598231117

In [25]:
from sklearn.model_selection import cross_val_score
num_folds = 5

# Perform k-fold cross-validation
scores = cross_val_score(model, inputs, targets, cv=num_folds, scoring="neg_mean_squared_error")

rmse_scores = np.sqrt(-scores)

# Print the RMSE scores for each fold
for i, rmse in enumerate(rmse_scores):
    print(f"Fold {i + 1} RMSE: {rmse:.2f}")

# Calculate the mean and standard deviation of RMSE scores
mean_rmse = rmse_scores.mean()
std_rmse = rmse_scores.std()

print(f"Mean speed: {targets.mean()}")
print(f"Std speed: {targets.std()}")
print(f"Mean RMSE: {mean_rmse:.2f}")
print(f"Std RMSE: {std_rmse:.2f}")

# Large values of cross-validation errors indicate multi-co-linearity (fix with Ridge?)

Fold 1 RMSE: 0.48
Fold 2 RMSE: 0.48
Fold 3 RMSE: 0.49
Fold 4 RMSE: 0.47
Fold 5 RMSE: 0.52
Mean speed: 15.116993941217087
Std speed: 1.5254575121564231
Mean RMSE: 0.49
Std RMSE: 0.02


In [26]:
test_with_predictions["PredictedRank"] = test_with_predictions.groupby("RaceId")["PredictedSpeed"].rank(method="dense", ascending=False)

In [27]:
(test_with_predictions[test_with_predictions["RaceId"] == 789029]
 [["RaceId", "HorseId", "HorseName", "FinishingPosition", "PredictedRank", "Speed", "PredictedSpeed", "HorseCount", "DecimalOdds"]]
 #.sort_values("PredictedRank")
 .head(200))

,RaceId,HorseId,HorseName,FinishingPosition,PredictedRank,Speed,PredictedSpeed,HorseCount,DecimalOdds
1183,789029,2951763,Chalk Stream,1,2.0,15.562294,15.006497,5,2.2
1184,789029,3142176,Barn Owl,2,3.0,15.506711,14.987300,5,3.5
1185,789029,3481885,Carrigillihy,3,1.0,15.457025,15.015428,5,4.5
1186,789029,3098714,Grantley,4,5.0,15.293678,14.683790,5,15.0
1187,789029,3098690,Chase The Dollar,5,4.0,15.144306,14.840632,5,15.0


In [36]:
correct_winners = test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] == 1)]

correct_winners_count = len(correct_winners)
incorrect_winners_count = len(test_with_predictions[(test_with_predictions["FinishingPosition"] == 1) & (test_with_predictions["PredictedRank"] != 1)])

winnings = correct_winners["DecimalOdds"].sum()

print(f"Correct winners {correct_winners_count}, Incorrect winners {incorrect_winners_count}")
print(f"Win percentage {100.0 * correct_winners_count / (correct_winners_count + incorrect_winners_count)}")
print(f"Winnings {winnings}, losses {incorrect_winners_count}, diff  {winnings - incorrect_winners_count}")

Correct winners 269, Incorrect winners 540
Win percentage 33.25092707045736
Winnings 898.2589396714396, losses 540, diff  358.25893967143963


In [29]:
# Base = 18.93845003737852
# PolynomialFeatures(degree=2, interaction_only=True) = 20.00996760528283
# With PolynomialFeatures(degree=2, interaction_only=False) = 20.08472464490406
# With rest days capped =  20.134562671318218
# with max 11 horses in race = 22.24711907810499 (£467)
# with max 8 horses in race = 24.011299435028246 (£336)
# with max 6 horses in race = 27.59562841530055 (£343)
# with max 5 horses in race = 32.330827067669176 (£269)
# with Ridge = 33.25092707045736 (£358)